# hposuite additional usage examples

In [2]:
# It is assumed that the datadir is hposuite/data

from hposuite.benchmarks import BENCHMARKS
from hposuite.optimizers import OPTIMIZERS

print(BENCHMARKS.keys())
print(OPTIMIZERS.keys())

dict_keys(['yahpo-lcbench-3945', 'yahpo-lcbench-7593', 'yahpo-lcbench-34539', 'yahpo-lcbench-126025', 'yahpo-lcbench-126026', 'yahpo-lcbench-126029', 'yahpo-lcbench-146212', 'yahpo-lcbench-167104', 'yahpo-lcbench-167149', 'yahpo-lcbench-167152', 'yahpo-lcbench-167161', 'yahpo-lcbench-167168', 'yahpo-lcbench-167181', 'yahpo-lcbench-167184', 'yahpo-lcbench-167185', 'yahpo-lcbench-167190', 'yahpo-lcbench-167200', 'yahpo-lcbench-167201', 'yahpo-lcbench-168329', 'yahpo-lcbench-168330', 'yahpo-lcbench-168331', 'yahpo-lcbench-168335', 'yahpo-lcbench-168868', 'yahpo-lcbench-168908', 'yahpo-lcbench-168910', 'yahpo-lcbench-189354', 'yahpo-lcbench-189862', 'yahpo-lcbench-189865', 'yahpo-lcbench-189866', 'yahpo-lcbench-189873', 'yahpo-lcbench-189905', 'yahpo-lcbench-189906', 'yahpo-lcbench-189908', 'yahpo-lcbench-189909', 'mfh3_bad', 'mfh6_bad', 'mfh3_good', 'mfh6_good', 'mfh3_moderate', 'mfh6_moderate', 'mfh3_terrible', 'mfh6_terrible', 'pd1-cifar100-wide_resnet-2048', 'pd1-imagenet-resnet-512', 

## Running multiple different optimizers on multiple different benchmarks

The following example runs RandomSearch and SMAC Blackbox Facade on Ackley Functional Benchmark and MF Hartmann 3D Synthetic Benchmark

In [ ]:
rs = OPTIMIZERS["RandomSearch"]
bb = OPTIMIZERS["SMAC_BO"]

ackley = BENCHMARKS["ackley"]
mfh3g = BENCHMARKS["mfh3_good"]

from hposuite import create_study

study = create_study(
    name="hposuite-2opts-2benchs",
    output_dir="example-outputs",
    optimizers=[rs, bb],
    benchmarks=[ackley, mfh3g],
    num_seeds=3,
    budget=10,
)

study.optimize(overwrite=True)

## Running the same optimizer with different hyperparameters

In this example we use SMAC_Hyperband with eta=2 and eta=3 and run it on MF Hartmann 6D Benchmark

In [ ]:
mf_hb_2 = (
    OPTIMIZERS["SMAC_Hyperband"],
    {"eta": 2}
)
mf_hb_3 = (
    OPTIMIZERS["SMAC_Hyperband"],
    {"eta": 3}
)

from hposuite import create_study

study = create_study(
    name="hposuite-1opt-2hps",
    output_dir="example-outputs",
    optimizers=[mf_hb_2, mf_hb_3],
    benchmarks="mfh6_bad",  # Benchmarks can also be specified as string names
    num_seeds=3,
    budget=10,
)

study.optimize(overwrite=True)

## Running one single-objective Optimizer on the same Benchmark for different objectives in the same study

For this example we select the YAHPO LCBench Surrogate Benchmark. \
Listing below the objectives (metrics and costs) available in the benchmark

In [6]:
lcbench_3945 = BENCHMARKS["yahpo-lcbench-3945"]
print(lcbench_3945.metrics.keys())
print(lcbench_3945.costs.keys())

dict_keys(['val_accuracy', 'val_cross_entropy', 'val_balanced_accuracy'])
dict_keys(['time'])


Running RandomSearch on the Benchmark with one objective at a time in a single study

In [ ]:
from hposuite import create_study

study = create_study(
    name="hposuite-1opt-1bench-2objs",
    output_dir="example-outputs",
    optimizers="RandomSearch",
    benchmarks=[
        (lcbench_3945, {"objectives": "val_cross_entropy"}),
        (lcbench_3945, {"objectives": "val_accuracy"}),
    ],
    num_seeds=3,
    budget=10,
)

study.optimize(overwrite=True)

## Running a Multi-objective Optimizer on the same benchmark multiple times with different pairs of objectives

It is possible to run an MO Optimizer, say Optuna-NSGA2 on an MO benchmark like YAHPO LCBench taking different combinations of its objectives at a time, within the same study. \
For example: One run on the pair (val_cross_entropy, val_accuracy) and another on (val_cross_entropy, time)

In [ ]:
from hposuite import create_study

study = create_study(
    name="hposuite-1opt-1bench-mo-mix",
    output_dir="example-outputs",
    optimizers="RandomSearch",
    benchmarks=[
        (lcbench_3945, {"objectives": ["val_cross_entropy", "val_accuracy"]}),
        (lcbench_3945, {"objectives": ["val_cross_entropy", "time"]}),
    ],
    num_seeds=3,
    budget=10,
)

study.optimize(overwrite=True)

## Running a Prior-based Optimizer on a Benchmark with multiple objectives, with priors on different objectives at a time.

For this example, we use RandomSearch With Priors as the Optimizer and PYMOO SymPart Problem as the benchmark. \
The example below demonstrates how to optimize on a benchmark with priors over one of its multiple objectives at a time, in a single study

In [3]:
rsp = OPTIMIZERS["RandomSearchWithPriors"]
sympart = BENCHMARKS["pymoo-sympart"]

Printing the Benchmark Description of the Pymoo ZDT1 Benchmark

In [4]:
sympart

BenchmarkDescription(name='pymoo-sympart', config_space=Configuration space object:
  Hyperparameters:
    x0, Type: UniformFloat, Range: [-100.0, 100.0], Default: 0.0
    x1, Type: UniformFloat, Range: [-100.0, 100.0], Default: 0.0
, load=functools.partial(<function _get_pymoo_problems at 0x7f5ad4b29750>, function_name='sympart'), metrics={'value1': Measure(minimize=True, kind=<Kind.METRIC: 'metric'>, bounds=(-inf, inf)), 'value2': Measure(minimize=True, kind=<Kind.METRIC: 'metric'>, bounds=(-inf, inf))}, test_metrics=None, costs=None, fidelities=None, has_conditionals=False, is_tabular=False, env=Env(name='py310-pymoo-0.6.1.3'), mem_req_mb=1024, predefined_points=None, extra={})

Running RandomSearchWithPriors on Pymoo Sympart with prior over one objective at a time

In [ ]:
from hposuite import create_study

study = create_study(
    name="hposuite-ex-multiprior-so",
    output_dir="example-outputs",
    optimizers=rsp,
    benchmarks=[
        (sympart, {
            "objectives": ["value1"],
            "priors": {
                "value1": {
                    "x0": 50.0,
                    "x1": 50.0,
                }
            }
        }),
        (sympart, {
            "objectives": ["value2"],
            "priors": {
                "value2": {
                    "x0": 20.0,
                    "x1": 30.0,
                }
            }
        }),
    ],
    num_seeds=3,
    budget=10,
    on_error="raise",
)

study.optimize(overwrite=True)

## Running a Prior-based Multi-objective Optimizer on a Multi-objective benchmark with priors over different objectives at a time

For this example, we again use RandomSearch With Priors as the Optimizer, with the hyperparameter mo-prior-sampling set to random, which picks which objective's prior to use at every iteration randomly. \
The PYMOO DTLZ1 Many Objective Problem is selected as the benchmark. \
The example below demonstrates how to optimize for Multiple Objectives on a benchmark with priors over one of a pair of its multiple objectives at a time, in a single study

In [3]:
rsp = OPTIMIZERS["RandomSearchWithPriors"]
dtlz1 = BENCHMARKS["pymoo-dtlz1"]

Let's display, pymoo-dtlz1's description

In [4]:
dtlz1

BenchmarkDescription(name='pymoo-dtlz1', config_space=Configuration space object:
  Hyperparameters:
    x0, Type: UniformFloat, Range: [0.0, 1.0], Default: 0.5
    x1, Type: UniformFloat, Range: [0.0, 1.0], Default: 0.5
    x2, Type: UniformFloat, Range: [0.0, 1.0], Default: 0.5
    x3, Type: UniformFloat, Range: [0.0, 1.0], Default: 0.5
    x4, Type: UniformFloat, Range: [0.0, 1.0], Default: 0.5
    x5, Type: UniformFloat, Range: [0.0, 1.0], Default: 0.5
    x6, Type: UniformFloat, Range: [0.0, 1.0], Default: 0.5
, load=functools.partial(<function _get_pymoo_problems at 0x7b5d66bf1750>, function_name='dtlz1'), metrics={'value1': Measure(minimize=True, kind=<Kind.METRIC: 'metric'>, bounds=(-inf, inf)), 'value2': Measure(minimize=True, kind=<Kind.METRIC: 'metric'>, bounds=(-inf, inf)), 'value3': Measure(minimize=True, kind=<Kind.METRIC: 'metric'>, bounds=(-inf, inf))}, test_metrics=None, costs=None, fidelities=None, has_conditionals=False, is_tabular=False, env=Env(name='py310-pymoo-0.

Generating a random prior for the pymoo-dtlz1 benchmark

In [5]:
import numpy as np

dtlz1_priors = {
    f"value{i}": {
        var.name: np.random.uniform(var.lower, var.upper) for var in list(dtlz1.config_space.values())
    }
    for i in range(1, 4)
}

from pprint import pprint
pprint(dtlz1_priors)

{'value1': {'x0': 0.19419993333048924,
            'x1': 0.6614779339684947,
            'x2': 0.544900305283605,
            'x3': 0.26440861964677975,
            'x4': 0.41235220818493157,
            'x5': 0.864046574915138,
            'x6': 0.3311259829012908},
 'value2': {'x0': 0.9798940978253005,
            'x1': 0.8443932203723541,
            'x2': 0.6604842848181971,
            'x3': 0.3554239597245046,
            'x4': 0.7319828518310841,
            'x5': 0.6029296320892957,
            'x6': 0.01626536051098082},
 'value3': {'x0': 0.3481083542516228,
            'x1': 0.30891397333194337,
            'x2': 0.815010110519983,
            'x3': 0.08701246224184911,
            'x4': 0.3681382403794782,
            'x5': 0.6872534330433554,
            'x6': 0.019535086267388646}}


Running RandomSearch With Priors on DTLZ1 optimizing for two objectives and taking priors over one of the objectives at a time

In [ ]:
from hposuite import create_study

study = create_study(
    name="hposuite-ex-multiprior-priors",
    output_dir="example-outputs",
    optimizers=rsp,
    benchmarks=[
        (
            dtlz1, {
                "objectives": ["value1", "value2"],
                "priors": {
                    "value1": dtlz1_priors["value1"]
                }
            }
        ),
        (
            dtlz1, {
                "objectives": ["value2", "value3"],
                "priors": {
                    "value2": dtlz1_priors["value2"]
                }
            }
        ),
        (
            dtlz1, {
                "objectives": ["value3", "value1"],
                "priors": {
                    "value3": dtlz1_priors["value3"]
                }
            }
        ),
    ],
    num_seeds=3,
    budget=10,
    on_error="raise",
)

study.optimize(overwrite=True)